# Data Governance Framework: Dynamic Views & Role-Based Access Control (RBAC)

## Objective
This notebook implements a tiered security model using **Dynamic Views**. Instead of modifying the underlying data, we project different "views" of the data based on the identity of the user running the query (`is_account_group_member()`).

## Security Policy Implementation

### 1. Row-Level Security (RLS)
We enforce a logic-based row filter to control *which* records are visible.
*   **Executives / Data Owners:** Full access to all history.
*   **Analysts:** Access restricted to validated data (e.g., records after `2025-10-01`) to ensure analysis is performed on stable, high-sample populations.

### 2. Column-Level Security (CLS)
We enforce data masking on sensitive columns.
*   **Logic:** The `cost_of_equity` column is returned as `NULL` for the `analysts` group but remains visible for leadership.

In [0]:
%sql
-- 1. Define the Security Logic Function
-- This function acts as the central policy definition for RLS.
CREATE OR REPLACE FUNCTION filter_gold_access(report_date DATE)
RETURN 
  CASE 
    -- Tier 1: Full Access
    WHEN is_account_group_member('executives') OR is_account_group_member('data_owners') THEN TRUE
    
    -- Tier 2: Restricted Access (Verified Data Only)
    WHEN is_account_group_member('analysts') THEN report_date >= to_date('2025-10-01')
    
    -- Tier 3: Default Deny
    ELSE FALSE
  END;

In [0]:
%sql
-- 2. Apply Security Policy to View
CREATE OR REPLACE VIEW secure_gold_analytics AS
SELECT 
    symbol,
    valuation_date,
    close_price,
    wacc,
    -- [CLS] Column Masking Logic
    CASE 
        WHEN is_account_group_member('analysts') THEN NULL  -- Mask
        ELSE cost_of_equity                                 -- Reveal
    END as cost_of_equity,
    momentum 
FROM 
    delta.`/Volumes/workspace/default/storage/serving/valuation_dashboard_v4`
WHERE 
  -- [RLS] Row Filtering Logic
  filter_gold_access(report_date) = TRUE;

In [0]:
%sql
-- 3. Documentation Metadata
COMMENT ON VIEW secure_gold_analytics IS 'Security View on Gold Volume. Policies: [RLS: Verified Data Only], [CLS: Cost of Equity masked for Analysts]';

In [0]:
%sql
-- 4. Grant Access Privileges
-- Users can now query the VIEW, but not the underlying table/volume directly.
GRANT SELECT ON VIEW secure_gold_analytics TO `analysts`;
GRANT SELECT ON VIEW secure_gold_analytics TO `executives`;

## Verification Matrix

| User Persona | Action | Expected Result |
| :--- | :--- | :--- |
| **Executive** | `SELECT *` | **Success:** Returns all rows, all columns unmasked. |
| **Analyst** | `SELECT *` | **Partial:** Returns filtered rows, `cost_of_equity` is NULL. |
| **Analyst** | `OPTIMIZE` | **Failure:** `Access Denied` (SELECT only). |

In [0]:
%sql
-- Audit Log Check
SELECT event_time, user_identity.email, service_name, action_name, response.error_message
FROM system.access.audit
ORDER BY event_time DESC;